In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import patsy
from matplotlib.pylab import *
import os
from os import path
from IPython.display import *
from sklearn import cross_validation
from sklearn import svm
from sklearn import linear_model
from scipy.sparse import linalg as la
from IPython.display import *
import tensorflow as tf
import multiprocessing as mp
from scipy import sparse
from scipy.sparse import linalg as sla

## User Based Filtering Development

For this analysis we want to attempt to define a mathematical model to predict the user's rating of unknown 
movies.  The first attempt will be to look at a simple user based filtering approach.  
This approach looks at correlations in ratings of movies 
between the ratings of individual users.  To start out we consider a simple model consisting of three 
terms:

* Average total purchase made by a user

* Average total purchase made at the store

* Latent factors which correlate user ratings and movies (This will be discussed more later)

To create this model we start with the following expression:

$$ P(User, Movie) = 
\alpha_{U} \times \overline{R(User)} + 
\alpha_{N} \times \overline{R(Movie)} + 
F_{SVD}(User, Movie| \lambda, N_{Rank}) $$

This expression defines a baseline purchase as simply the mean purchase amount at each store
, $\overline{R(Movie)}$, in a weighted average with the mean purchase amount at a customer
, $\overline{R(User)}$.  The term, $F_{SVD}(User, Movie| \lambda, N_{Rank})$
is a derived from Funk's SVD decomposition, used win Netflix's recommender system prize 
(This will be discussed in more detail shortly).

In [4]:
## Get the data
source_file = 'ExampleData/ml-1m.zip'
if not os.path.isdir("ExampleData"):
    os.mkdir("ExampleData")
if not os.path.isfile("ExampleData/ml-1m.zip"):
    from urllib.request import urlretrieve
    urlretrieve('http://files.grouplens.org/datasets/movielens/ml-1m.zip', source_file)
import zipfile
fid = zipfile.ZipFile(source_file,'r')
fid.extractall('ExampleData')

In [5]:
data_directory = 'ExampleData/ml-1m/'
df_ratings = pd.read_csv(path.join(data_directory, 'ratings.dat'), header=None, sep="::")
df_ratings.columns = ['user_id', 'movie_id', 'rating', 'time']

# If any movie or user has fewer than 20 entries drop them
min_count = 20
user_counts = df_ratings['user_id'].value_counts() 
user_ids = user_counts[user_counts > min_count].index
movie_counts = df_ratings['movie_id'].value_counts()
movie_ids = movie_counts[movie_counts > min_count].index

movie_ok = df_ratings.movie_id.isin(movie_ids)
user_ok = df_ratings.user_id.isin(user_ids)
all_ok = movie_ok & user_ok

df_ratings = df_ratings[all_ok].copy()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [6]:
HTML(df_ratings.head(n=10).to_html())

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


## User and Movie Average Ratings

Prehaps the most niave model is to simply look at the recommendation as the mean rating 
given by a user and given to a movie.  This simple approach has several appeals:

1) Both the average movie rating and the average user rating can be measured with relatively small samples (<50)

2) The number of free parameters is small, reducing the risk of over training

3) Easily fit and stable to new observations

This first section will show only fitting these two parameters.

Performance will be assessed using 70/30 training and validation split.

### Algrebraic Manipulation

The formula shown earlier has an implicit constraints:

$$ \sum{\alpha_{i}} = 1 $$

While there are toolkits which handle these constraints, sklearn's 
linear model fitting is unfortunately not one of them.  
But we back into these constraints with some simple linear algrebra:

$$ P(User, Movie) - \overline{P_{User}}  = \alpha_{Movie}
\left(\overline{P_{Movie}} - \overline{P_{User}}\right) $$

This means we will only be fitting for $\alpha_{Movie}$ and will back into $\alpha_{User} = 1-\alpha_{Movie}$

In [7]:
# Measure the mean ratings for per user and per movie
def GetMeanRatings(df):
    mean_user_rating = df.groupby("user_id")['rating'].mean()
    mean_user_rating.name = 'mean_user_rating'
    mean_movie_rating = df.groupby("movie_id")['rating'].mean()
    mean_movie_rating.name = 'mean_movie_rating'
    return(mean_user_rating, mean_movie_rating)

# Append the mean ratings to a data frame
def AppendMeanRatings(df, mean_user_ratings, mean_movie_ratings):
    df_augment = df.copy()
    df_augment['mean_user_rating'] = mean_user_ratings.loc[df.user_id].values
    df_augment['mean_movie_rating'] = mean_movie_ratings.loc[df.movie_id].values
    return(df_augment)

# Create X, y for the linear model
def BuildPredictionModel(df):
    X = df.mean_movie_rating - df.mean_user_rating
    y = df.rating - df.mean_user_rating
    X = X.values.reshape(-1, 1)
    return(X, y)

# Predict the ratings (y) from a trained linear model
def PredictModel(df, mdl):
    X, _ = BuildPredictionModel(df)
    delta_p = mdl.predict(X)
    predict_rating = df.mean_user_rating + delta_p
    return(predict_rating)

# Run the training and prediction model
def TrainAndPredict(df_train, df_test):
    mean_user_ratings, mean_movie_ratings = GetMeanRatings(df_train)
    df_train_aug = AppendMeanRatings(df_train, mean_user_ratings, mean_movie_ratings)
    df_test_aug = AppendMeanRatings(df_test, mean_user_ratings, mean_movie_ratings)
    Xtrain, ytrain = BuildPredictionModel(df_train_aug)
    mdl = linear_model.LinearRegression(fit_intercept=False)
    mdl.fit(Xtrain, ytrain)
    ypred = PredictModel(df_test_aug, mdl)
    err = ypred - df_test_aug.rating
    rmse = np.sqrt((err**2).mean())
    return(rmse, mdl)

# Execute the training/validation split
def SplitTrainingValidation(df, training_percentage = 0.70):
    split = cross_validation.ShuffleSplit(df.shape[0], n_iter=1, test_size=1-training_percentage)
    train_idx, test_idx = list(split)[0]
    train_index = df.index[train_idx]
    test_index = df.index[test_idx]
    df_train = df.loc[train_index].copy()
    user_ids = df_train.user_id.drop_duplicates()
    movie_ids = df_train.movie_id.drop_duplicates()
    df_test = df.loc[test_index].copy()
    df_test = df_test[df_test.movie_id.isin(movie_ids) & 
                     df_test.user_id.isin(user_ids)].copy()
    return(df_train, df_test)


In [8]:
## Look at simple model performance as a baseline
df_train, df_test = SplitTrainingValidation(df_ratings)
rmse, mdl = TrainAndPredict(df_train, df_test)
print("RMSE = %f, alpha_{Movie}=%f"%(rmse, mdl.coef_[0]))

RMSE = 0.952301, alpha_{Movie}=0.634819


## Augmenting Model with Latent Factors

The winning Netflix algorithm start began by representing the movie rating matrix in the 
form $R_{Users, Movie}$, where the rows are the users and the columns are the movies, 
[citation](http://www.scielo.br/pdf/jistm/v13n3/1807-1775-jistm-13-03-0497.pdf).  If 
all users had rated all movies this matrix would be fulled measured.  In that case 
it would be reasonable to decompose the matrix using the familiar signular value decomposition (SVD), 
$R_{Users, Movies} = U \Sigma V$, where $U$ and $V$ are square with the number of users and number
of movies as the number of columns, respectively.  The columns of $U$ would represent linear
combinations of correlated users and the rows of $V$ would represent correlated movies.  If 
$R_{Users, Movies}$ was full know we could use the SVD to reduce the dimensionality of the space
using a Principal Component Analysis to approximate $R_{Users, Movies}$ as 
only a product and identify highly correlated movies or users.  Unfortunately we 
do not have a fully determined $R_{Users, Movies}$.

Funk's algorithm instead took the available data we do have and used it to fit the following model:

$$ R_{Users, Movies} = R_{Users, N} \times R_{Movies, N}^{T} $$

Where $N$ is the number of latent basis vectors in the model, and all terms in 
both  $R_{Users, N}$ and $R_{Movies, N}$ are unknown.  This means for a problem 
like this example data we only have ~5.5% of the elements of $R_{Users, Movies}$.  This means
we need to start the problem with some approximations for the SVD.  Also rather than fitting 
the ratings themselves, we will instead fit the residuals of the
linear model prediction,

The first we need a starting point for the algorithm's convergence.  To do this we will 
run a SVD assuming $R_{Users, Movies}$ is fully determined, where $R_{Users, Movies}$ 
is the residual.  To do this we will look at two 
cases:

1) If $R_{Users, Movies}$ is known (e.g. in the training data) residual between the linear 
model and the observed rating

2) If $R_{Users, Movies}$ is unknown set it to zero



In [9]:
mean_user_ratings, mean_movie_ratings = GetMeanRatings(df_train)
df_train_aug = AppendMeanRatings(df_train, mean_user_ratings, mean_movie_ratings)
predicted_rating = PredictModel(df_train_aug, mdl)
df_train_aug['predicted_rating'] = predicted_rating

# Measure the residual for the training data
df_train_aug['residual'] = df_train_aug.predicted_rating - df_train_aug.rating

# Make a pivot table for the observed values
tbl = df_train_aug.pivot_table(values='residual', columns = 'movie_id', 
                     index = 'user_id', aggfunc='sum', fill_value=0)
# Transform the pivot table into a sparse matrix
mtx = sparse.coo_matrix(tbl.as_matrix())

# Solve for the leading (N=30) SVD values
n_components = 30
U, S, V = la.svds(mtx, k=n_components)

# Set $R_{Users, N}$ to U \time S^{1/2}
# Set $R_{Movies, N}$ to V \time S^{1/2}
df_users = pd.DataFrame(U.dot(np.diag(np.sqrt(S))), index=tbl.index)
df_movies = pd.DataFrame(np.diag(np.sqrt(S)).dot(V).T, index=tbl.columns)
df_users.columns =  ['ULatent_%s'%str(ii).zfill(2) for ii in range(n_components)]
df_movies.columns = ['MLatent_%s'%str(ii).zfill(2) for ii in range(n_components)]

Now we can execute the following minimization:

$$ Minimize~((R_{Users, Movies} - R_{Users, N} \times R_{Movies, N}^{T}))^{2} + 
\lambda \left(||R_{Users, N}|| + ||R_{Movies, N}|| \right) $$

This first expression is simply the Mean Absolute Error of the prediction.  The 
second expression is a regularization parameter, used to contrain the size of either 
$R_{Users, N}$ or $R_{Movies, N}$.

Practically we will simply fix $R_{Users, N}$ and fit $R_{Movies, N}$.  Once that 
converges we will fix $R_{Movies, N}$ to the optimal value and fit $R_{Users, N}$.

In [10]:

def FitLatentWeights(latent_row, index, dependent_column,
                     df_complimentary, df_grp, complimentary_column, alpha):
    df_sub = df_grp.get_group(index)
    X = df_complimentary.loc[df_sub[complimentary_column]]
    y = df_sub[dependent_column]
    mdl = linear_model.SGDRegressor(loss='squared_loss', penalty='l1', alpha =alpha, fit_intercept=False, 
                                   n_iter = 100)
    mdl.fit(X.as_matrix(), y.as_matrix())
    coefs = mdl.coef_
    df = pd.Series(coefs, index=latent_row.index.values)
    return(df)

def PredictSVDRating(df, df_user_latent, df_movie_latent, predicted_rating_column):
    p1 = df_user_latent.loc[df.user_id]
    p2 = df_movie_latent.loc[df.movie_id]
    p_prod = (p1.values * p2.values).sum(axis=1)
    prediction = df[predicted_rating_column]-p_prod
    return(prediction)

In [11]:
lambda_value =1e-2
df_user_grp = df_train_aug.groupby('user_id')
df_movie_grp = df_train_aug.groupby("movie_id")
df_users_latent = df_users.apply(lambda row: FitLatentWeights(row, row.name, 'residual',
                                                              df_movies, 
                                                              df_user_grp, 'movie_id', lambda_value), axis=1)
df_movie_latent = df_movies.apply(lambda row: FitLatentWeights(row, row.name, 'residual',
                                                               df_users_latent, 
                                                               df_movie_grp, 
                                                               'user_id', lambda_value), axis=1)

In [12]:
df_test_aug = AppendMeanRatings(df_test, mean_user_ratings, mean_movie_ratings)
predicted_rating = PredictModel(df_test_aug, mdl)
df_test_aug['predicted_rating'] = predicted_rating
df_test_aug['SVD Prediction'] = PredictSVDRating(df_test_aug, df_users_latent, df_movie_latent, 'predicted_rating')

In [13]:
# Look at RMSE for the model augmented with the SVD
err= df_test_aug['SVD Prediction'] - df_test_aug.rating
print("RMSE = %f"%np.sqrt((err**2).mean()))

RMSE = 0.866730


## Results

Other recommender systems such as [Surpriselib](http://surpriselib.com) show smalled RMSEs than this (~0.70).  This 
is in part due to sklearn's Stochastic Gradient Solver using the
the squared loss function $((R_{Users, Movies} - R_{Users, N} \times R_{Movies, N}^{T}))^{2}$, as 
opposed to the Mean Absolute Error used by Supriselib.  
In order to use this loss function either use Supriselib or
code the algorithm in Tensorflow, which allows more flexible loss function definitions. 

We can make these changes going forward, but this shows a working framework.

## Bringing it All Together

Back to the original expression:


$$ P(User, Movie) = 
\alpha_{U} \times \overline{R(User)} + 
\alpha_{N} \times \overline{R(Movie)} + 
F_{SVD}(User, Movie| \lambda, N_{Rank}) $$

This has two free parameters which can be tuned: $\lambda$ and $N_{Rank}$.  $\lambda$ controls how large SVD any row 
of the SVD matrix become and $N_{Rank}$ is the rank of the system.  We would like to optimize these two parameters.

I did some manual tuning and found a good parameter set of $N_{Rank} = 30$ and $\lambda = 1e-2$.
The class below is a self contained object which executes all the stages 
of the recommender system.

In [14]:
class RecommenderSVD:
    def __init__(self, loss_lambda, Nrank, 
                 rating_column = 'rating', user_column = 'user_id', 
                item_column = 'movie_id'):
        self.loss_lambda = loss_lambda
        self.Nrank = Nrank
        self.rating_column = rating_column
        self.user_column = user_column
        self.item_column = item_column
        self.linear_model = linear_model.LinearRegression(fit_intercept=False)
        self.latent_mdl = linear_model.SGDRegressor(loss='squared_loss', penalty='l1', 
                                                    alpha =self.loss_lambda, fit_intercept=False,
                                                   n_iter=100)


    def train(self, df):
        df_train = df.copy()
        self.assign_group_means(df_train)
        df_train['mean_user_rating'] = self.mean_user_rating.loc[df_train[self.user_column]].values
        df_train['mean_item_rating'] = self.mean_item_rating.loc[df_train[self.item_column]].values
        
        ## Fit the linear model
        self.fit_linear_model(df_train)
        df_train['linear_prediction'] = self.predict_linear_model(df_train)

        ## Fit the SVD model
        df_train['linear_residuals'] = df_train['linear_prediction'] - df_train[self.rating_column]
        self.fit_svd_model(df_train)
        
        
    def predict(self, df):
        df_test = df.copy()
        df_test['mean_user_rating'] = self.mean_user_rating.loc[df_test[self.user_column]].values
        df_test['mean_item_rating'] = self.mean_item_rating.loc[df_test[self.item_column]].values
        
        ## Predict the linear model
        df_test['linear_prediction'] = self.predict_linear_model(df_test)     
        
        ## Predict svd model
        df_test['svd_prediction'] = self.predict_svd_model(df_test)
        return(df_test['svd_prediction'])
    def fit_linear_model(self, df_train):
        X = df_train.mean_item_rating - df_train.mean_user_rating
        y = df_train[self.rating_column] - df_train.mean_user_rating
        X = X.values.reshape(-1, 1)
        self.linear_model.fit(X, y)
    
    def fit_svd_model(self, df):
        tbl = df.pivot_table(values='linear_residuals', columns = self.item_column, 
                             index = self.user_column, aggfunc='sum', fill_value=0)
        mtx = sparse.coo_matrix(tbl.as_matrix())

        n_components = self.Nrank
        U, S, V = la.svds(mtx, k=n_components)

        df_users = pd.DataFrame(U.dot(np.diag(np.sqrt(S))), index=tbl.index)
        df_items = pd.DataFrame(np.diag(np.sqrt(S)).dot(V).T, index=tbl.columns)
        df_users.columns =  ['ULatent_%s'%str(ii).zfill(2) for ii in range(n_components)]
        df_items.columns = ['MLatent_%s'%str(ii).zfill(2) for ii in range(n_components)]        
        df_user_grp = df.groupby(self.user_column)
        df_item_grp = df.groupby(self.item_column)
        self.df_users_latent = df_users.apply(lambda row: self.fit_svd_latent_weights(row, row.name, 
                                                                                      'linear_residuals',
                                                                                      df_items, df_user_grp, 
                                                                                      self.item_column), axis=1)
        self.df_item_latent = df_items.apply(lambda row: self.fit_svd_latent_weights(row, 
                                                                                     row.name, 'linear_residuals',
                                                                                     self.df_users_latent, 
                                                                                     df_item_grp, 
                                                                                     self.user_column), axis=1)
    def fit_svd_latent_weights(self, latent_row, index, dependent_column,
                               df_complimentary, df_grp, complimentary_column):
        df_sub = df_grp.get_group(index)
        X = df_complimentary.loc[df_sub[complimentary_column]]
        y = df_sub[dependent_column]
        self.latent_mdl.fit(X.as_matrix(), y.as_matrix())
        coefs = np.array([cx for cx in self.latent_mdl.coef_])
        df_results = pd.Series(coefs, index=latent_row.index.values)
        return(df_results)


    def predict_linear_model(self, df):
        X = df.mean_item_rating - df.mean_user_rating
        X = X.values.reshape(-1,1)
        delta_p = self.linear_model.predict(X)
        predict_rating = df.mean_user_rating + delta_p
        return(predict_rating)
    
    def predict_svd_model(self, df):
        p1 = self.df_users_latent.loc[df[self.user_column]]
        p2 = self.df_item_latent.loc[df[self.item_column]]
        p_prod = (p1.values * p2.values).sum(axis=1)
        prediction = df['linear_prediction']-p_prod
        return(prediction)

    def group_means(self, df, column_name):
        mean_vector = df.groupby(column_name)[self.rating_column].mean()
        return(mean_vector)
    
    def assign_group_means(self, df):
        self.mean_user_rating = self.group_means(df, self.user_column)
        self.mean_item_rating = self.group_means(df, self.item_column)

In [15]:
mdl_svd = RecommenderSVD(1e-2, 30)
mdl_svd.train(df_train)
prediction = mdl_svd.predict(df_test)
err = df_test['rating'] - prediction
rmse = np.sqrt((err**2).mean())
print("RMSE = %f"%rmse)

RMSE = 0.866730


In [16]:
rounded_prediction = np.round(prediction)
rounded_prediction[rounded_prediction < df_test.rating.min()] = df_test.rating.min()
rounded_prediction[rounded_prediction > df_test.rating.max()] = df_test.rating.max()

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion = pd.DataFrame(confusion_matrix(df_test.rating, rounded_prediction), 
                         columns = ['Predition = %i'%ii for ii in range(1, 6)], 
                         index= ['Observed = %i'%ii for ii in range(1, 6)])
HTML(confusion.to_html())

,Predition = 1,Predition = 2,Predition = 3,Predition = 4,Predition = 5
Observed = 1,1678,6085,6743,2033,50
Observed = 2,459,6491,17521,7175,140
Observed = 3,138,4456,38226,34000,919
Observed = 4,25,1114,24758,73217,4900
Observed = 5,8,180,5528,47168,14932


## Areas for improvement

* **sklearn's** optimization routine is effective, but has some limitation in defining loss 
functions and optimization routines.  Other tools such as **Tensorflow** give more flexiblity, 
but are not very inituative.  

* Content based filter: This only looks at correlations between user's and movie's.  
Content based filtering brings in more content focused results.  The data 
tends to be categorical in nature, so there is some processing which needs to 
take place.  Tools like **pasty** are very useful in transforming these data types
